<a href="https://colab.research.google.com/github/GonMazzini/Loads_Surrogate_Transferability/blob/main/BaseModel_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

% Saved 13/3/2022    
- This notebook contains the implementation of the "baseline" model. Additionally, the model and the tensors are converted to cuda to allow running on gpu. 

In [1]:
import pandas as pd 
import numpy as np
import math
from random import shuffle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU")
else:
    device = torch.device("cpu")
    print("CPU")

GPU


# 0- Read the data as a data frame

In [3]:
df = pd.read_excel('LoadsDataBase_6D_Set123_FiltMinMaxCrit.xlsx') # Average the values from Set1,Set2 and Set3.
df.head(2)
# 0 : TT_Mx_avg
# 1 : TT_My_avg
# 2 : TB_Mx_avg
# 3 : TB_My_avg
# 4 : MS_Mz_avg
# 5 : BR_Mx_avg
# 6 : BR_My_avg
# 7 : TT-Mz_avg

,Unnamed: 0,pointno,U,SigmaU,Alpha,MannL,MannGamma,VeerDeltaPhi,TT_Mx_avg,TT_My_avg,TB_Mx_avg,TB_My_avg,TT_Mz_avg,MS_Mz_avg,BR_Mx_avg,BR_My_avg
0,0,1,4.000000,0.100000,-0.650000,7.500000,1.000000,-22.250000,747.561872,200.666288,6708.717789,8861.885588,819.209904,63.457528,4253.317748,15006.726860
1,1,2,10.150758,1.208656,-0.139692,48.470634,1.363636,-4.771217,3556.031457,676.339081,16692.647572,6329.099515,3746.460605,1354.995442,10409.290476,16289.414152


# ============================================================
# ==========================Section 1 ==========================
# ============================================================

In [4]:
X = df.iloc[:,2:8]
y = df.iloc[:,8:]

In [5]:
# Test split:
X, X_test, y, y_test = train_test_split(X,y, test_size = 0.2, shuffle = True,  random_state = 101)

print(f'The filtered data set consits on: {len(df)} entries.')
print(f'A total of {len(X)} will be used for training and validation.')
print(f'A total of {len(X_test)} will be used for testing the final model.')




The filtered data set consits on: 7664 entries.
A total of 6131 will be used for training and validation.
A total of 1533 will be used for testing the final model.


### From now on, "X" and "y" will be used for train-validate the model. 

In [6]:
feature_range = (0, 1)
scaler_x = preprocessing.MinMaxScaler(feature_range=feature_range).fit(X)
X_scaled = scaler_x.transform(X)

### Separte between train and validation

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled,y.values, test_size = 0.2, shuffle = True,  random_state = 101)

# printing number of samples for train-validation-test
print(f'A total of {y_train.shape[0]} for training, {round(100*y_train.shape[0]/len(df),1)} % of total data')
print(f'A total of {y_val.shape[0]} for validation, {round(100*y_val.shape[0]/len(df),1)} % of total data')
print(f'A total of {y_test.shape[0]} for testing, {round(100*y_test.shape[0]/len(df),1)} % of total data')

A total of 4904 for training, 64.0 % of total data
A total of 1227 for validation, 16.0 % of total data
A total of 1533 for testing, 20.0 % of total data


### The PyTorch worfklow can be summarized as follows:
- 1) Design model (input, output size, forward pass)
- 2) Construct loss and optimizer
- 3) Training loop
   - forward pass: compute prediction based on the current weights and biases of the net
   - backward pass: compute the gradients of the loss function wrt. to model parameters
   - update weigths in

In [8]:
input_size = 6             # np.shape(X_train)[1]
output_channels = 8        # np.shape(y_train)[1]
num_hid_1 = 50
num_hid_2 = 50

In [9]:
class Net(nn.Module):
    
    def __init__(self, num_hid_1, num_hid_2):   
        super(Net,self).__init__()  # inherit from the superclass Module
        self.num_hid_1 = num_hid_1
        self.num_hid_2 = num_hid_2
        
        self.fc1 = nn.Linear(in_features= input_size,
                             out_features= self.num_hid_1,                             
                            bias = True)  
        
        self.fc2 = nn.Linear(in_features = self.num_hid_1, 
                             out_features = self.num_hid_2,
                            bias = True)
        
        self.fc3 = nn.Linear(in_features = self.num_hid_2, 
                             out_features = output_channels,
                            bias = True)
        
#        self.dropout = nn.Dropout(p=0.15)
        
    def forward(self,x):
        
        out = self.fc1(x)  
#        out = self.dropout(out)
        out = F.relu(out)
        out = self.fc2(out)
#        out = self.dropout(out)
        out = F.relu(out)
        out = self.fc3(out)                       #  torch.tanh(self.fc3(out))
        
        return out  

In [10]:
# instantiate the class
model = Net(num_hid_1,num_hid_2)

# if torch.cuda.is_available():
#   model.to(device)

# Define the loss function (mean square error)
loss = nn.MSELoss()

# Instantiate optimizer passing the net parameters as argument, and learning rate
optimizer = optim.Adam(model.parameters(), lr = 0.01)

# list to store results
train_losses , val_losses= [],[]



In [11]:
# Try a dummy forward
#model(torch.tensor(X_train[0]).float().to(device))  # beware that need to convert from double to float

# 2.1- Use the PyTorch DataLoader and Dataset utils.
- DataLoader class combines a dataset and a sampler, and provides an iterable over the given dataset for training the model
- Dataset: just an abstract class representing a :class:`Dataset`

In [12]:
class FatigueLoads_TrainSet(Dataset):

    def __init__(self):
        self.n_samples = X_train.shape[0]
        self.x_data = torch.from_numpy(X_train) # size [n_samples, n_features]
        self.y_data = torch.from_numpy(y_train) # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    
class FatigueLoads_ValidationSet(Dataset):

    def __init__(self):
        self.n_samples = X_val.shape[0]
        self.x_data = torch.from_numpy(X_val) # size [n_samples, n_features]
        self.y_data = torch.from_numpy(y_val) # size [n_samples, 1]

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

### Get first sample and unpack. Note that the enviromental inputs are normalized using MinMaxScaler

In [13]:
train_dataset = FatigueLoads_TrainSet()
valid_dataset = FatigueLoads_ValidationSet()

first_data = train_dataset[0]
features, loads = first_data
print(features, loads)

tensor([0.9213, 0.5395, 0.2971, 0.1574, 0.7369, 0.3075], dtype=torch.float64) tensor([ 7116.1376,   808.7441, 21966.9989, 16383.7768,  7347.3186,   807.4632,
        18567.6736, 16935.8620], dtype=torch.float64)


In [22]:
batch_size = X_val.shape[0]//8
num_epochs = 8

num_batches_train = X_train.shape[0] // batch_size
num_batches_test = X_val.shape[0] // batch_size

print(f'Num batches train: {num_batches_train}')
print(f'Num batches valid: {num_batches_test}')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

Num batches train: 32
Num batches valid: 8


# 3.1- Start training loop 

In [15]:
ErrorOnPUPOSE

NameError: ignored

In [16]:
torch.cuda.memory_allocated(device)

0

In [ ]:
15360.00 + 560640.00 
# batch: 0, Memory After forward 576000.00 MB
# batch: 0, Memory consuming by forward 560640.00 MB

In [23]:
%%time

print(f'initial memory {torch.cuda.memory_allocated(device)/(1024*1024*8):.4f}')
# /(1024*1024*8)
if torch.cuda.is_available():
  model.to(device)
  print(f' memory after device {torch.cuda.memory_allocated(device)/(1024*1024*8):.4f}')
# /(1024*1024*8)
for epoch in range(num_epochs):
    
    model.train()
    print(f'Begin epoch: {epoch}')
    for i, (inputs, loads) in enumerate(train_loader):
        #print(features, loads)
        
        optimizer.zero_grad()                      # zeroize accumulated gradients in parameters 
        a = torch.cuda.memory_allocated(device)/(1024*1024*8) #/(1024*1024*8)            
        print(f'batch: {i}, memory Before forward {a:.2f} MB')
        output = model(inputs.float().to(device))             # forwards pass       
        b = torch.cuda.memory_allocated(device)/(1024*1024*8) #/(1024*1024*8)   
        print(f'batch: {i}, Memory After forward {b:.2f} MB')
        c = (b-a) # NO NEED TO DIVIDE again
        print(f'batch: {i}, Memory consuming by forward {c:.4f} MB')
        batch_loss = loss(output, loads.float().to(device))   # compute loss for current batch
        
        batch_loss.backward()                      # compute the gradient of the loss wrt. model parameters
        d = torch.cuda.memory_allocated(device)/(1024*1024*8) #/(1024*1024*8)  
        print(f'batch: {i}, Memory After backward {d:.4f} MB')
        optimizer.step()                           # update weights according to the comptued gradients
        e = torch.cuda.memory_allocated(device)/(1024*1024*8) #/(1024*1024*8) 
        print(f'batch: {i}, Memory After optimizer step {e:.4f} MB')
    
    epoch_loss_train = 0
    epoch_loss_test = 0
    model.eval()
    
    ##### Evaluate training
    for i, (inputs, loads) in enumerate(train_loader):
        
        output = model(inputs.float().to(device))
        
        batch_loss_train = loss(output, loads.float().to(device))  # compute loss for the current batch
        epoch_loss_train += batch_loss_train            # accumulate loss for the current epoch
        
        #print(f'Epoch: {epoch+1}/{num_epochs}  | Step {i+1}/{n_iterations}')
    
    ##### Evaluate validation    
    for i, (inputs, loads) in enumerate(valid_loader):
        
        output = model(inputs.float().to(device))
        
        batch_loss_test = loss(output, loads.float().to(device))  # compute loss for the current batch
        epoch_loss_test += batch_loss_test     # accumulate loss for the current epoch
        
        #print(f'Epoch: {epoch+1}/{num_epochs}  | Step {i+1}/{n_iterations}')
    
    if epoch % 1 == 0: 
        print(f'Epoch: {epoch+1}/{num_epochs} | Train loss: {epoch_loss_train/num_batches_train}       | Val loss {epoch_loss_test/num_batches_test}')
        
    # store in list for plotting the loss per epoch    
    val_losses.append(epoch_loss_test.cpu().detach().numpy()/num_batches_test)  
    train_losses.append(epoch_loss_train.cpu().detach().numpy()/num_batches_train)  

initial memory 0.3688
 memory after device 0.3688
Begin epoch: 0
batch: 0, memory Before forward 0.37 MB
batch: 0, Memory After forward 0.38 MB
batch: 0, Memory consuming by forward 0.0084 MB
batch: 0, Memory After backward 0.3695 MB
batch: 0, Memory After optimizer step 0.3695 MB
batch: 1, memory Before forward 0.37 MB
batch: 1, Memory After forward 0.38 MB
batch: 1, Memory consuming by forward 0.0078 MB
batch: 1, Memory After backward 0.3695 MB
batch: 1, Memory After optimizer step 0.3695 MB
batch: 2, memory Before forward 0.37 MB
batch: 2, Memory After forward 0.38 MB
batch: 2, Memory consuming by forward 0.0078 MB
batch: 2, Memory After backward 0.3695 MB
batch: 2, Memory After optimizer step 0.3695 MB
batch: 3, memory Before forward 0.37 MB
batch: 3, Memory After forward 0.38 MB
batch: 3, Memory consuming by forward 0.0078 MB
batch: 3, Memory After backward 0.3695 MB
batch: 3, Memory After optimizer step 0.3695 MB
batch: 4, memory Before forward 0.37 MB
batch: 4, Memory After forw

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(train_losses[50:])
plt.plot(val_losses[50:], '-k')
plt.title('Training-Validation loss', fontsize = 16)
plt.grid()
plt.legend(['Training loss','Validation loss'])

# Evalute on the test data

### Scale the test data before evaluating the model

In [ ]:
Test_scaler_x = preprocessing.MinMaxScaler(feature_range=feature_range).fit(X_test) # maybe try another for X?
X_Test_scaled = Test_scaler_x.transform(X_test)

predictions = model(torch.tensor(X_Test_scaled).float().to(device)).cpu().detach().numpy()
test_targets = y_test.values

In [ ]:
for idx, ch in enumerate(df.columns.tolist()[8:]):
  # issues with 2 and 7
    plt.figure(idx, figsize = (8,6))
    sns.scatterplot(x = predictions[:,idx], y = test_targets[:,idx])
    plt.title(f'DELs: {ch}', fontsize = 20)
    plt.xlabel('Predictions [kNm]', fontsize = 12)
    plt.ylabel('Targets [kNm]')

In [ ]:
mse_list = []
r2_score_list = []
for i in range(len(df.columns.tolist()[8:])):
    #print(f'MSE {AllTargetData.columns[i]} Channel : \n {mean_squared_error(AllTargetData.values[:,i], Yout[:,i])}')
    mse_list.append(mean_squared_error(test_targets[:,i], predictions[:,i]))
    r2_score_list.append(r2_score(test_targets[:,i], predictions[:,i]))
 

 # Compute the normalized mean square error:
Norm_RMSE = np.sqrt(np.array(mse_list)) / y_test.describe().loc['mean'].values

In [ ]:
fig = plt.figure(figsize= (10,4)) 
sns.barplot(x= df.columns.tolist()[8:], y= r2_score_list) 
plt.ylim([min(r2_score_list)*0.92,max(r2_score_list)*1.02])
plt.ylabel('R2 score', fontsize = 20)
plt.grid()

In [ ]:
fig = plt.figure(figsize= (10,4)) 
sns.barplot(x= df.columns.tolist()[8:], y= Norm_RMSE) 
plt.ylim([0,max(Norm_RMSE)*1.02])
plt.ylabel('Normalized Root Mean Square Error', fontsize = 20)
plt.grid()

# ============================================================
# ==========================Section 2 ==========================
# ============================================================
### Compare versus the wind2loads neural net

In [ ]:
error on purpose

In [ ]:
from w2l import neuralnets

In [ ]:
# define the model using the same net architecture, and train for the same number of epochs using the previously batch size as well.
w2l_net = neuralnets.ann(layersizes = [input_size, num_hid_1, num_hid_2, output_channels],
                       params = {'minibatchsize':64, 'nepochs':500}, 
                       output_style = 'None',
                        testratios = [0.7, 0.3, 0.])

In [ ]:
# train using the data from the first split
Outdata = w2l_net.train(X.values,y.values)

In [ ]:
w2l_net.params

In [ ]:
Outdata.keys()

$\color{red}{\text{Why 500 epochs tho}}$

In [ ]:
plt.plot(Outdata['Jhist'][50:])


In [ ]:
Yout = w2l_net.predict(X_test.values) 

In [ ]:
for idx, ch in enumerate(df.columns.tolist()[8:]):
  # issues with 2 and 7
    plt.figure(idx, figsize = (8,6))
    sns.scatterplot(Yout[:,idx],test_targets[:,idx])
    sns.scatterplot(predictions[:,idx],test_targets[:,idx])
    plt.title(f'DELs: {ch}', fontsize = 20)
    plt.legend(['W2L','PyTorch'])
    plt.xlabel('Predictions [kNm]', fontsize = 12)
    plt.ylabel('Targets [kNm]')

In [ ]:
W2L_mse_list = []
W2L_r2_score_list = []
for i in range(len(df.columns.tolist()[8:])):
    #print(f'MSE {AllTargetData.columns[i]} Channel : \n {mean_squared_error(AllTargetData.values[:,i], Yout[:,i])}')
    W2L_mse_list.append(mean_squared_error(test_targets[:,i], Yout[:,i]))
    W2L_r2_score_list.append(r2_score(test_targets[:,i], Yout[:,i]))
    
W2L_Norm_RMSE = np.sqrt(np.array(W2L_mse_list)) / y_test.describe().loc['mean'].values

In [ ]:
fig = plt.figure(figsize= (12,6)) 
sns.barplot(x= df.columns.tolist()[8:], y= W2L_r2_score_list) 
plt.ylim([min(W2L_r2_score_list)*0.92,max(W2L_r2_score_list)*1.02])
plt.ylabel('R2 score', fontsize = 20)
plt.grid()

# Compare Wind2Loads net versus PyTorch implementation

In [ ]:
barplot_lst = []
for i,ch in enumerate(df.columns.tolist()[8:]):
    barplot_lst.append(['pytorch',ch,r2_score_list[i]])
for i,ch in enumerate(df.columns.tolist()[8:]):
    barplot_lst.append(['W2L',ch,W2L_r2_score_list[i]])  

In [ ]:
df_comparison = pd.DataFrame(barplot_lst,
                  columns=['Model','channel','r2'])

In [ ]:
plt.figure(figsize=(12,6))
df_comparison.pivot("channel", "Model", "r2").plot(kind='bar')
plt.ylim([min(r2_score_list)*0.98,max(r2_score_list)*1.02])
plt.title('R2', fontsize = 18)
plt.show()

In [ ]:
barplot_lst_NMSE = []
for i,ch in enumerate(df.columns.tolist()[8:]):
    barplot_lst_NMSE.append(['pytorch',ch , Norm_RMSE[i]])
for i,ch in enumerate(df.columns.tolist()[8:]):
    barplot_lst_NMSE.append(['W2L', ch, W2L_Norm_RMSE[i]])

In [ ]:
df_NRMSE_comparison = pd.DataFrame(barplot_lst_NMSE,
                  columns=['Model','channel','NRMSE'])



In [ ]:
plt.figure(figsize=(14,6))
df_NRMSE_comparison.pivot("channel", "Model", "NRMSE").plot(kind='bar')
plt.ylim([0,max(W2L_Norm_RMSE)*1.02])
plt.show()